In [43]:
## Python code for numerically optimising an input image to yield a maximum class score
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os
import scipy.io
from scipy import misc
import matplotlib.pyplot as plt

sys.path.insert(0, '../')
import visualising_vgg as vgg

In [2]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_DEPTH = 3
path = '../../Tensorflow-CS20SI/Assignment_2/style_transfer/imagenet-vgg-verydeep-19.mat'
class_arg = 100 ## Visualising 100th class
LEARNING_RATE = 1e-3

In [3]:
## Loading the graph
print 'Loading the graph..'
vgg_model = scipy.io.loadmat(path)
vgg_layers = vgg_model['layers']

Loading the graph..


In [4]:
def _fully_connected(vgg_layers, prev_layer, layer, expected_layer_name):
    
    with tf.variable_scope(expected_layer_name) as scope:

        W, b = vgg._weights(vgg_layers, layer, expected_layer_name)
        W = tf.constant(W)
        b = tf.constant(b)
        try:
            w, x, y, z = W.shape
            W = tf.reshape(W, [int(w*x*y), int(z)])
        except:
            pass
        try:
            w, x, y, z = prev_layer.shape
            flattened = int(x*y*z)
        except:
            flattened = int(prev_layer.shape[1])
            
        flat = tf.reshape(prev_layer, [-1,flattened])
        full = tf.matmul(flat, W)
        relu = tf.nn.relu(full + b)
        return relu

In [5]:
def _dropout(prev_layer, keep_probs, expected_layer_name):
    
    with tf.variable_scope(expected_layer_name) as scope:
        return tf.nn.dropout(prev_layer, keep_probs)

In [6]:
def loss(image, score, reg_param):
    with tf.variable_scope("loss") as scope:
        loss = tf.reduce_mean(score - reg_param*tf.reduce_sum(tf.square(image)))
        return loss

In [7]:
def _optimizer(loss):
    with tf.variable_scope('optimizer') as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(-1*loss)
        return optimizer

### Class Model Visualisation

In [29]:
tf.reset_default_graph()

In [30]:
with tf.variable_scope("Inputs") as scope:
    image = tf.Variable(initial_value=tf.random_normal(shape=[1, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH]),name="Image")
    keep_probs = tf.placeholder(dtype=tf.float32,name="keep_probs")

In [31]:
## Assembling the graph
graph = {} 
graph['conv1_1']  = vgg._conv_relu(vgg_layers, image, 0, 'conv1_1')
graph['conv1_2']  = vgg._conv_relu(vgg_layers, graph['conv1_1'], 2, 'conv1_2')
graph['avgpool1'] = vgg._avg_pool(graph['conv1_2'],'avgpool1')
graph['conv2_1']  = vgg._conv_relu(vgg_layers, graph['avgpool1'], 5, 'conv2_1')
graph['conv2_2']  = vgg._conv_relu(vgg_layers, graph['conv2_1'], 7, 'conv2_2')
graph['avgpool2'] = vgg._avg_pool(graph['conv2_2'],'avgpool2')
graph['conv3_1']  = vgg._conv_relu(vgg_layers, graph['avgpool2'], 10, 'conv3_1')
graph['conv3_2']  = vgg._conv_relu(vgg_layers, graph['conv3_1'], 12, 'conv3_2')
graph['conv3_3']  = vgg._conv_relu(vgg_layers, graph['conv3_2'], 14, 'conv3_3')
graph['conv3_4']  = vgg._conv_relu(vgg_layers, graph['conv3_3'], 16, 'conv3_4')
graph['avgpool3'] = vgg._avg_pool(graph['conv3_4'],'avgpool3')
graph['conv4_1']  = vgg._conv_relu(vgg_layers, graph['avgpool3'], 19, 'conv4_1')
graph['conv4_2']  = vgg._conv_relu(vgg_layers, graph['conv4_1'], 21, 'conv4_2')
graph['conv4_3']  = vgg._conv_relu(vgg_layers, graph['conv4_2'], 23, 'conv4_3')
graph['conv4_4']  = vgg._conv_relu(vgg_layers, graph['conv4_3'], 25, 'conv4_4')
graph['avgpool4'] = vgg._avg_pool(graph['conv4_4'],'avgpool4')
graph['conv5_1']  = vgg._conv_relu(vgg_layers, graph['avgpool4'], 28, 'conv5_1')
graph['conv5_2']  = vgg._conv_relu(vgg_layers, graph['conv5_1'], 30, 'conv5_2')
graph['conv5_3']  = vgg._conv_relu(vgg_layers, graph['conv5_2'], 32, 'conv5_3')
graph['conv5_4']  = vgg._conv_relu(vgg_layers, graph['conv5_3'], 34, 'conv5_4')
graph['avgpool5'] = vgg._avg_pool(graph['conv5_4'],'avgpool5')
graph['fc6'] = _fully_connected(vgg_layers, graph['avgpool5'], 37, 'fc6')
graph['dropout1'] = _dropout(graph['fc6'],keep_probs,'dropout1')
graph['fc7'] = _fully_connected(vgg_layers, graph['dropout1'], 39, 'fc7')
graph['dropout2'] = _dropout(graph['fc7'], keep_probs, 'dropout2')
graph['fc8'] = _fully_connected(vgg_layers, graph['dropout2'], 41, 'fc8')

In [32]:
graph['score'] = graph['fc8'][0,class_arg]
graph['loss'] = loss(image, graph['score'], reg_param=0.1)

In [33]:
graph['optimizer'] = _optimizer(graph['loss'])

In [34]:
N_EPOCHS = 20

In [35]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(N_EPOCHS):
        
        _, l, score = sess.run([graph['optimizer'], graph['loss'], graph['score']], 
                       feed_dict = {keep_probs:0.5})
        print 'Epoch: {}\tLoss: {}\tScore: {}'.format(i+1, l, score)

Epoch: 1	Loss: -150530.078125	Score: 0.0
Epoch: 2	Loss: -150289.828125	Score: 0.0
Epoch: 3	Loss: -150050.015625	Score: 0.0705909729004
Epoch: 4	Loss: -149810.40625	Score: 0.0
Epoch: 5	Loss: -149571.234375	Score: 0.0
Epoch: 6	Loss: -149332.265625	Score: 0.0
Epoch: 7	Loss: -149093.75	Score: 0.0
Epoch: 8	Loss: -148855.515625	Score: 0.0
Epoch: 9	Loss: -148617.59375	Score: 0.0
Epoch: 10	Loss: -148380.03125	Score: 0.0
Epoch: 11	Loss: -148142.71875	Score: 0.0
Epoch: 12	Loss: -147905.609375	Score: 0.336689054966
Epoch: 13	Loss: -147669.03125	Score: 0.380449056625
Epoch: 14	Loss: -147433.21875	Score: 0.0
Epoch: 15	Loss: -147197.359375	Score: 0.0
Epoch: 16	Loss: -146961.921875	Score: 0.0
Epoch: 17	Loss: -146726.859375	Score: 0.0
Epoch: 18	Loss: -146492.125	Score: 0.0
Epoch: 19	Loss: -146257.84375	Score: 0.0
Epoch: 20	Loss: -146023.828125	Score: 0.0
